## Json Parser
This notebook consumes **Delta CDF** from `submissions` where `status='content_extraction_completed'`, calls a **parse_json_content unstructured workflow**

## Imports

## ConfigManager

In [ ]:
"""
Configuration management utilities for PDF processing framework.

This module provides utilities for loading, validating, and managing
configuration files used throughout the PDF processing pipeline.
"""

import yaml
import os
import time
from typing import Dict, Any, List, Optional, Union
from pathlib import Path
from pyspark.sql import Window
from pyspark.sql import functions as F
import pandas as pd
import traceback


class ConfigManager:
    """
    Configuration manager for handling YAML and JSON configuration files.
    
    Provides methods for:
    - Loading configuration from files
    - Validating configuration structure
    - Merging configurations
    - Environment variable substitution
    """
    
    def __init__(self, config_path: Optional[Union[str, Path]] = None):
        """
        Initialize configuration manager.
        
        Args:
            config_path: Path to configuration file
            config_dict: Configuration dictionary (alternative to file)
            
        Raises:
            ValueError: If neither config_path nor config_dict is provided
        """
        if config_path:
            self.config = self.load_config_file(config_path)
            self.config_path = str(config_path)
        else:
            raise ValueError("Either config_path or config_dict must be provided")
        
        # Substitute environment variables
        # self._substitute_environment_variables()

    @staticmethod
    def get_file_extension(file_path: Union[str, Path]) -> str:
        """
        Get file extension (without dot).
        
        Args:
            file_path: Path to file
            
        Returns:
            File extension in lowercase
        """
        return Path(file_path).suffix.lower().lstrip('.')
    
    @staticmethod
    def load_config_file(file_path: Union[str, Path]) -> Dict[str, Any]:
        """
        Load configuration from YAML or JSON file.
        
        Args:
            file_path: Path to configuration file
            
        Returns:
            Configuration dictionary
            
        Raises:
            FileNotFoundError: If file doesn't exist
            ValueError: If file format is not supported
        """
        extension = ConfigManager.get_file_extension(file_path)
        
        if extension in ['yaml', 'yml']:
            with open(file_path, 'r', encoding='utf-8') as f:
                return yaml.safe_load(f) or {}
        else:
            raise ValueError(f"Unsupported configuration file format: {extension}")
    
    def get(self, key: str, default: Any = None) -> Any:
        """
        Get configuration value with optional default.
        
        Args:
            key: Configuration key (supports dot notation for nested keys)
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        return self._get_nested_value(self.config, key, default)
        
    @staticmethod
    def _get_nested_value(config: Dict[str, Any], key: str, default: Any = None) -> Any:
        """
        Get nested configuration value using dot notation.
        
        Args:
            config: Configuration dictionary
            key: Key with dot notation (e.g., 'task.params.value')
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        keys = key.split('.')
        value = config
        
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        
        return value
        
    def to_dict(self) -> Dict[str, Any]:
        """
        Get configuration as dictionary.
        
        Returns:
            Configuration dictionary
        """
        return self.config.copy()
    

    def get_pipeline_steps(config: dict, pipeline_name: str, document_format: str, step_name: str) -> dict:
        """
        Return the steps dictionary for a given pipeline and document format.
        """
        step_list = config.get(pipeline_name, {}).get(document_format, [])
        step_config = ConfigManager.get_step_config(step_list, step_name)
        return step_config


    @staticmethod
    def get_step_config(config: list, step_name: str) -> dict:
        """
        Fetch all variables for a given step from the workflow config.

        Args:
            config (list): List of step dictionaries.
            step_name (str): Step name to search for.

        Returns:
            dict: Config dictionary for the step, or {} if not found.
        """
        for step in config:
            if step.get("step") == step_name:
                return step
        return {}

   


## Get the task variables required for the workflow

In [ ]:
# ---------------- CONFIGURATION ----------------

import json
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType, IntegerType, BooleanType, ArrayType, DoubleType

environment = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="environment")
job_id = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="databricks_job_id")
job_run_id = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="databricks_job_run_id")

# Load configuration sections that were processed by ConfigManager in unified_setup
globals_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="globals_config"))
services_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="services_config"))
pipelines_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="pipelines_config"))

# Scalability parameters
# num_partitions = xmlparser_cfg["num_partitions"]
max_files_per_trigger = globals_config["default_batch_size"]
# Table configuration using ConfigManager-processed values
catalog = globals_config["catalog"]
bronze_schema_name = globals_config["bronze_schema_name"]
silver_schema_name = globals_config["silver_schema_name"]
# Streaming configuration for scalability
checkpoint_base_location = globals_config["checkpoint_base_location"]
checkpoint_location = f"{checkpoint_base_location}/json_parsing_checkpoints"

jsonparser_cfg = ConfigManager.get_pipeline_steps(
    pipelines_config, "unstructured", "pdf", "JsonParser")
output_submission_status = jsonparser_cfg["submission_status"]
submissions_table = submission_table = f"{catalog}.{bronze_schema_name}.{jsonparser_cfg["submissions_table"]}"
document_payloads_table = f"{catalog}.{silver_schema_name}.{jsonparser_cfg["document_payloads_table"]}"
output_table = f"{catalog}.{silver_schema_name}.{jsonparser_cfg["output_table"]}"

synthetic_mode = jsonparser_cfg["synthetic_mode"]
micro_batch_limit = jsonparser_cfg["micro_batch_limit"]
target_partitions = jsonparser_cfg["target_partitions"]


print(" Configuration loaded:")
print(f"    submissions_table: {submissions_table}")
print(f"    output_table: {output_table}")
print(f"    Checkpoint: {checkpoint_location}")
print(f"    Max files per trigger: {max_files_per_trigger}")
print(f"    Synthetic mode: {synthetic_mode}")
print(f"    Job ID: {job_id}, Run ID: {job_run_id}")

result_schema = StructType([
    StructField("submission_id", StringType(), False),
    StructField("node_type", StringType(), True),
    StructField("node_id", StringType(), True),
    StructField("node_page", IntegerType(), True),
    StructField("node_bounding_box", ArrayType(ArrayType(DoubleType())), True),
    StructField("node_parent", StringType(), True),
    StructField("node_content", StringType(), True),
    StructField("image_base64encoded_string", StringType(), True),
    StructField("text_as_html", StringType(), True),
    StructField("error_traceback", StringType(), True),
    StructField("parsing_status", StringType(), True),
    StructField("processing_time_ms", LongType(), True),
    StructField("parsing_timestamp", TimestampType(), True),                
])

In [ ]:
# Initialize framework step executor for XML parsing
# Create step executor instance with synthetic mode flag
# step_executor = StepExecutor(use_synthetic=synthetic_mode) 

# print("✅ Using XML parsing from framework StepExecutor")
# print(f"   - Synthetic mode: {synthetic_mode}")
# print("   - Framework provides parse_xml_content method")
# print("   - Scalable and reusable across workflows")



## Read Source Data
Read data from submissions table for submission_status=content_extraction_completed

In [ ]:

# CDF-based streaming source for automatic change detection
source_stream = (
    spark.readStream
         .format("delta")
         .option("readChangeFeed", "true")  # Enable CDF
         .table(submissions_table)
         .where(F.col("_change_type").isin("insert", "update_postimage"))
         .where(F.col("submission_status") == "content_extraction_completed")
         .selectExpr("* EXCEPT(_change_type, _commit_version, _commit_timestamp)")
)

submissions_stream = source_stream.filter(F.col("pipeline_id") == "unstructured")

print(" CDF streaming source configured")
print("   - Automatically detects status changes")
print("   - Event-driven processing (no polling)")
print("   - Exactly-once processing guarantees")


### process_partition

In [ ]:
def json_to_dataframe_safe(json_path):
    """Handle missing nested keys without errors"""
    
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
    except Exception as e:
        return {
            "parsing_status": "error",
            "error_message": f"error in reading the json file {json_path}",
            "error_traceback": f"{traceback.format_exc()}"
        }
    
    rows = []

    try:
        for item in data:
            row = {
                'node_type': item.get('type'),
                'node_id': item.get('element_id'),
                'node_content': item.get('text'),
                'node_bounding_box': item.get('metadata', {}).get('coordinates', {}).get('points'),
                'node_page': item.get('metadata', {}).get('page_number'),
                'node_parent': item.get('metadata', {}).get('parent_id'),
                'image_base64encoded_string': item.get('metadata', {}).get('image_base64'),
                'text_as_html': item.get('metadata', {}).get('text_as_html')
            }
            rows.append(row)
    except Exception as e:
        return {
            "parsing_status": "error",
            "error_message": f"error in getting keys from the json file {json_path}",
            "error_traceback": f"{traceback.format_exc()}"
        }
    
    try:
        df_content = pd.DataFrame(rows)

        # remove any records where the type is UncategorizedText and content is < 100
        # this only adds noise IMO
        df_content = df_content.loc[
            ~(
                (df_content["node_type"] == "UncategorizedText")
                & (df_content["node_content"].str.len() < 100)
            )
        ]
    except Exception as e:
        return {
            "parsing_status": "error",
            "error_message": f"error in formatting content {json_path}",
            "error_traceback": f"{traceback.format_exc()}"
        } 
    
    return {
        "parsing_status": "success",
        "data": df_content
    }

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, LongType, TimestampType
)
import pandas as pd
from datetime import datetime
import json


def process_partition(rows_iter):
    """
    mapInPandas iterator. For each pandas DataFrame (a partition batch),
    call your parse_xml_content() row by row and yield a pandas DataFrame
    matching output_schema.
    """
    for row_df in rows_iter:
        cols = result_schema.names
        now = datetime.now()
        if row_df.empty:
            yield pd.DataFrame(columns=cols)
            continue

        for _, row in row_df.iterrows():
            start_time = time.time()
            submission_id = row["submission_id"]
            download_path = row["download_path"]
            
            try:
                json_data = json_to_dataframe_safe(download_path)

                if json_data['parsing_status'] == "success":

                    processing_time_ms = int((time.time() - start_time) * 1000)
                    table = json_data["data"]

                    # add columns to standardize
                    table['submission_id'] = submission_id
                    if "error_traceback" not in table.columns.tolist():
                        table['error_traceback'] = None
                    if "parsing_status" not in table.columns.tolist():
                        table['parsing_status'] = 'success'
                    if "processing_time_ms" not in table.columns.tolist():
                        table['processing_time_ms'] = processing_time_ms
                    if "parsing_timestamp" not in table.columns.tolist():
                        table['parsing_timestamp'] = now
                else:
                    table = pd.DataFrame({
                        "submission_id": [submission_id],
                        "node_type": [None],
                        "node_id": [None],
                        "node_page": [None],
                        "node_bounding_box": [None],
                        "node_parent": [None],
                        "node_content": [None],
                        "image_base64encoded_string": [None],
                        "text_as_html": [None],
                        "error_traceback": [json_data['error_traceback']],
                        "parsing_status": [json_data['parsing_status']],
                        "processing_time_ms": [None],
                        "parsing_timestamp": [now]
                    })
            except Exception as e:
                table = pd.DataFrame({
                    "submission_id": [submission_id],
                    "node_type": [None],
                    "node_id": [None],
                    "node_page": [None],
                    "node_bounding_box": [None],
                    "node_parent": [None],
                    "node_content": [None],
                    "image_base64encoded_string": [None],
                    "text_as_html": [None],
                    "error_traceback": [traceback.format_exc()],
                    "parsing_status": ["error"],
                    "processing_time_ms": [None],
                    "parsing_timestamp": [now]
                })

        yield table


## foreach_batch

In [ ]:
def foreach_batch(batch_df, batch_id):
    """
    Serverless-safe: uses DataFrame API + mapInPandas (no RDDs), and set-based SQL to update statuses.
    Expects globals: source_tbl, document_payloads_table, target_tbl, num_partitions, job_id, job_run_id
    """
    if batch_df.isEmpty():
        print(f"→ Batch {batch_id}: empty")
        return

    print(f"executing {batch_df.count()} records in {batch_id}")

    spark = batch_df.sparkSession

    # Deduplicate by latest per submission_id inside the microbatch 
    w = Window.partitionBy("submission_id").orderBy(
        F.col("last_updated").desc_nulls_last())
    latest = (batch_df
              .withColumn("rn", F.row_number().over(w))
              .where("rn = 1")
              .drop("rn"))

    # Optional claim step to prevent parallel runs from double-processing:
    # Create a temp view and atomically mark a slice as 'claimed' by this job.
    latest.createOrReplaceTempView("micro_latest")

    limit_clause = f"LIMIT {int(micro_batch_limit)}" if micro_batch_limit else "" 
    spark.sql(f"""
      MERGE INTO {submission_table} t
      USING (
        SELECT submission_id
        FROM micro_latest
        WHERE submission_status IN ('content_extraction_completed')
        {limit_clause}
      ) c
      ON t.submission_id = c.submission_id
      WHEN MATCHED THEN UPDATE SET
        t.submission_status = 'json_parsing_in_progress',
        t.last_updated      = current_timestamp()
    """)

    print(f"print updated statuses on {submission_table}")

    # Load only the rows claimed by this run id (ensures single consumer per row)
    claimed = (spark.read.table(submission_table)
               .where(F.col("submission_status") == "json_parsing_in_progress"))

    claimed_enriched_df = (
        claimed.alias("b")
        .join(F.broadcast(spark.table(document_payloads_table).alias("p")),
              on=F.col("b.submission_id") == F.col("p.submission_id"),
              how="inner")
        .select(
            F.col("p.download_path"),
            F.col("b.submission_id"),
            F.col("b.file_path")
        )   
    )

    if claimed_enriched_df.count() == 0:
        print(f"→ Batch {batch_id}: No rows claimed")
        return

    # Repartition for wide fan-out, but keep skew keys (submission_id) as the partition column
    claimed_enriched_df = claimed_enriched_df.repartition(target_partitions, F.col("submission_id"))

    # MapPartitions on executors to call REST & write files
    result_df = claimed_enriched_df.mapInPandas(process_partition, schema=result_schema)
    print("map in pandas process complete!")
    result_df = result_df.withColumn("databricks_parsing_job_id", F.lit(job_id))
    result_df = result_df.withColumn("databricks_parsing_job_run_id", F.lit(job_run_id))
    result_df = result_df.withColumn("parsing_batch_id", F.lit(batch_id).cast("string"))
    result_df.write.format("delta").mode("append").saveAsTable(output_table)

    # update the status of the submission
    result_df.createOrReplaceTempView("json_results_batch")
    # MERGE results back, clear claim, set result metadata
    # success
    spark.sql(f"""
        UPDATE {submissions_table} AS s
        SET    s.submission_status = '{output_submission_status}',
                s.last_updated = current_timestamp()
        WHERE EXISTS (
        SELECT 1
        FROM json_results_batch r
        WHERE r.submission_id = s.submission_id
            AND r.parsing_status = 'success'
        )
    """)

    # failed
    spark.sql(f"""
        UPDATE {submissions_table} AS s
        SET    s.submission_status = 'json_parsing_failed',
                s.last_updated = current_timestamp()
        WHERE EXISTS (
        SELECT 1
        FROM json_results_batch r
        WHERE r.submission_id = s.submission_id
            AND r.parsing_status = 'error'
        )
    """)



## Write Data 
1. Write data to silver table
2. On success, update submission_status=xml_parsing_completed

In [ ]:
print("🚀 Starting scalable XML parsing streaming job...")

streaming_query = (submissions_stream.writeStream
                   .foreachBatch(foreach_batch)
                   .option("checkpointLocation", checkpoint_location)
                   .option("maxFilesPerTrigger", max_files_per_trigger)
                   .trigger(availableNow=True)  # Process all available data then stop
                   .outputMode("append")
                   .start())

print("XML Parsing Job Completed started")

try:
    streaming_query.awaitTermination()
    print("XML Parsing Job Completed completed successfully")    
    
except Exception as e:
    print(f"Streaming job error: {str(e)}")
    streaming_query.stop()
    raise
